In [1]:
import pandas as pd
import numpy as np

In [2]:
!pip install imbalanced-learn

Defaulting to user installation because normal site-packages is not writeable

[notice] A new release of pip is available: 24.1.2 -> 24.2
[notice] To update, run: python3 -m pip install --upgrade pip


In [3]:
df = pd.read_csv('cleaned_data.csv')

In [4]:
df.head()

,Unnamed: 0,date,price,address,bedrooms,bathrooms,parking,area,median_price,population,average_age,zipcode,state
0,0,29 04 24,480000,"1307/60 A'beckett Street, Melbourne",2,1,0,53.0,5501000,47279,0,3000,Victoria
1,1,27 03 24,815000,"1103/108 Flinders Street, Melbourne",2,2,1,93.0,5501000,47279,0,3000,Victoria
2,2,29 02 24,3350000,"3801/35 Spring Street, Melbourne",3,2,2,175.0,10721000000,47279,0,3000,Victoria
3,3,29 02 24,3350000,"3801/35 Spring Street, Melbourne",3,2,2,175.0,10721000000,47279,0,3000,Victoria
4,4,8 12 23,590000,"508/181 Exhibition Street, Melbourne",2,1,1,80.0,5501000,47279,0,3000,Victoria


In [5]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 54851 entries, 0 to 54850
Data columns (total 13 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   Unnamed: 0    54851 non-null  int64  
 1   date          54851 non-null  object 
 2   price         54851 non-null  int64  
 3   address       54851 non-null  object 
 4   bedrooms      54851 non-null  int64  
 5   bathrooms     54851 non-null  int64  
 6   parking       54851 non-null  int64  
 7   area          54851 non-null  float64
 8   median_price  54851 non-null  int64  
 9   population    54851 non-null  int64  
 10  average_age   54851 non-null  int64  
 11  zipcode       54851 non-null  int64  
 12  state         54851 non-null  object 
dtypes: float64(1), int64(9), object(3)
memory usage: 5.4+ MB


In [6]:
df.corr()

/tmp/ipykernel_10671/1134722465.py:1: FutureWarning: The default value of numeric_only in DataFrame.corr is deprecated. In a future version, it will default to False. Select only valid columns or specify the value of numeric_only to silence this warning.
  df.corr()


,Unnamed: 0,price,bedrooms,bathrooms,parking,area,median_price,population,average_age,zipcode
Unnamed: 0,1.000000,-0.066827,0.177255,-0.009244,0.027352,0.002345,0.027797,-0.100853,0.101792,0.411447
price,-0.066827,1.000000,0.309295,0.426080,0.011766,0.002356,0.558588,0.131060,-0.081190,-0.299358
bedrooms,0.177255,0.309295,1.000000,0.507762,0.035953,0.017322,0.248636,0.032390,0.134987,0.007798
bathrooms,-0.009244,0.426080,0.507762,1.000000,0.019326,0.012059,0.258647,0.045800,0.023539,-0.111766
parking,0.027352,0.011766,0.035953,0.019326,1.000000,0.002687,0.003022,-0.008667,0.004558,0.019774
area,0.002345,0.002356,0.017322,0.012059,0.002687,1.000000,-0.007693,-0.005291,0.010960,0.002740
median_price,0.027797,0.558588,0.248636,0.258647,0.003022,-0.007693,1.000000,0.095193,-0.112824,-0.261424
population,-0.100853,0.131060,0.032390,0.045800,-0.008667,-0.005291,0.095193,1.000000,-0.120237,-0.375967
average_age,0.101792,-0.081190,0.134987,0.023539,0.004558,0.010960,-0.112824,-0.120237,1.000000,0.102529
zipcode,0.411447,-0.299358,0.007798,-0.111766,0.019774,0.002740,-0.261424,-0.375967,0.102529,1.000000


In [7]:
df['state'].value_counts()

New South Wales        17092
Victoria               11359
 Western Australia      9228
Queensland              7642
Tasmania                5689
 Southern Australia     2548
 Northern Territory     1293
Name: state, dtype: int64

In [8]:
from imblearn.over_sampling import SMOTE
from sklearn.preprocessing import LabelEncoder

df['state'] = df['state'].str.strip()

categorical_columns = df.select_dtypes(include=['object']).columns
categorical_columns = categorical_columns.drop('state')

df_encoded = df.copy()
label_encoders = {}
for col in categorical_columns:
    le = LabelEncoder()
    df_encoded[col] = le.fit_transform(df[col])
    label_encoders[col] = le


X = df_encoded.drop(columns=['state'])
y = df_encoded['state']

le_state = LabelEncoder()
le_state.fit(df['state'])
y_encoded = le_state.transform(df['state'])

smote = SMOTE(random_state=42)
X_resampled, y_resampled = smote.fit_resample(X, y_encoded)

y_resampled_decoded = le_state.inverse_transform(y_resampled)

resampled_df = pd.DataFrame(X_resampled, columns=X.columns)
resampled_df['state'] = y_resampled_decoded

df = resampled_df

df['state'].value_counts()

Victoria              17092
New South Wales       17092
Queensland            17092
Western Australia     17092
Southern Australia    17092
Northern Territory    17092
Tasmania              17092
Name: state, dtype: int64

In [9]:
df.corr()

/tmp/ipykernel_10671/1134722465.py:1: FutureWarning: The default value of numeric_only in DataFrame.corr is deprecated. In a future version, it will default to False. Select only valid columns or specify the value of numeric_only to silence this warning.
  df.corr()


,Unnamed: 0,date,price,address,bedrooms,bathrooms,parking,area,median_price,population,average_age,zipcode
Unnamed: 0,1.000000,0.325139,-0.110709,-0.001255,0.229514,-0.061193,0.027810,-0.001324,-0.010783,-0.161653,0.188616,0.324916
date,0.325139,1.000000,-0.052815,-0.013194,0.037817,-0.036581,0.013697,-0.001413,-0.014823,-0.087050,0.045004,0.116245
price,-0.110709,-0.052815,1.000000,0.020425,0.327488,0.410754,0.008221,0.000298,0.571312,0.189466,-0.059013,-0.137209
address,-0.001255,-0.013194,0.020425,1.000000,-0.001438,0.008112,-0.012875,0.010695,0.022099,-0.008232,0.008512,0.000680
bedrooms,0.229514,0.037817,0.327488,-0.001438,1.000000,0.466813,0.028391,0.013507,0.268576,0.033853,0.119991,0.079794
bathrooms,-0.061193,-0.036581,0.410754,0.008112,0.466813,1.000000,0.011580,0.003510,0.250267,0.062966,-0.019783,-0.135273
parking,0.027810,0.013697,0.008221,-0.012875,0.028391,0.011580,1.000000,0.001480,0.001210,-0.010469,0.001469,0.013912
area,-0.001324,-0.001413,0.000298,0.010695,0.013507,0.003510,0.001480,1.000000,-0.006878,-0.011117,0.003764,-0.008602
median_price,-0.010783,-0.014823,0.571312,0.022099,0.268576,0.250267,0.001210,-0.006878,1.000000,0.150832,-0.054784,-0.122927
population,-0.161653,-0.087050,0.189466,-0.008232,0.033853,0.062966,-0.010469,-0.011117,0.150832,1.000000,-0.117998,-0.178637


In [10]:
df

,Unnamed: 0,date,price,address,bedrooms,bathrooms,parking,area,median_price,population,average_age,zipcode,state
0,0,4881,480000,7286,2,1,0,53.000000,5501000,47279,0,3000,Victoria
1,1,4586,815000,4610,2,2,1,93.000000,5501000,47279,0,3000,Victoria
2,2,4857,3350000,26527,3,2,2,175.000000,10721000000,47279,0,3000,Victoria
3,3,4857,3350000,26527,3,2,2,175.000000,10721000000,47279,0,3000,Victoria
4,4,771,590000,32788,2,1,1,80.000000,5501000,47279,0,3000,Victoria
...,...,...,...,...,...,...,...,...,...,...,...,...,...
119639,36382,2513,311483,3149,1,1,1,259.033661,3271225,10397,0,6140,Western Australia
119640,55537,5436,564462,28615,4,2,2,756.364736,5101000,20294,0,6110,Western Australia
119641,69527,3482,415619,39765,4,1,1,1012.000000,4991000,2099,0,6430,Western Australia
119642,37198,3188,342638,20651,3,1,1,759.552000,3951000,13625,0,6112,Western Australia


In [11]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 119644 entries, 0 to 119643
Data columns (total 13 columns):
 #   Column        Non-Null Count   Dtype  
---  ------        --------------   -----  
 0   Unnamed: 0    119644 non-null  int64  
 1   date          119644 non-null  int64  
 2   price         119644 non-null  int64  
 3   address       119644 non-null  int64  
 4   bedrooms      119644 non-null  int64  
 5   bathrooms     119644 non-null  int64  
 6   parking       119644 non-null  int64  
 7   area          119644 non-null  float64
 8   median_price  119644 non-null  int64  
 9   population    119644 non-null  int64  
 10  average_age   119644 non-null  int64  
 11  zipcode       119644 non-null  int64  
 12  state         119644 non-null  object 
dtypes: float64(1), int64(11), object(1)
memory usage: 11.9+ MB


In [12]:
df

,Unnamed: 0,date,price,address,bedrooms,bathrooms,parking,area,median_price,population,average_age,zipcode,state
0,0,4881,480000,7286,2,1,0,53.000000,5501000,47279,0,3000,Victoria
1,1,4586,815000,4610,2,2,1,93.000000,5501000,47279,0,3000,Victoria
2,2,4857,3350000,26527,3,2,2,175.000000,10721000000,47279,0,3000,Victoria
3,3,4857,3350000,26527,3,2,2,175.000000,10721000000,47279,0,3000,Victoria
4,4,771,590000,32788,2,1,1,80.000000,5501000,47279,0,3000,Victoria
...,...,...,...,...,...,...,...,...,...,...,...,...,...
119639,36382,2513,311483,3149,1,1,1,259.033661,3271225,10397,0,6140,Western Australia
119640,55537,5436,564462,28615,4,2,2,756.364736,5101000,20294,0,6110,Western Australia
119641,69527,3482,415619,39765,4,1,1,1012.000000,4991000,2099,0,6430,Western Australia
119642,37198,3188,342638,20651,3,1,1,759.552000,3951000,13625,0,6112,Western Australia


In [13]:
le_state = LabelEncoder()

In [14]:
df['state'] = le_state.fit_transform(df['state'])

In [15]:
le_state.classes_

array(['New South Wales', 'Northern Territory', 'Queensland',
       'Southern Australia', 'Tasmania', 'Victoria', 'Western Australia'],
      dtype=object)

In [16]:
df.head()

,Unnamed: 0,date,price,address,bedrooms,bathrooms,parking,area,median_price,population,average_age,zipcode,state
0,0,4881,480000,7286,2,1,0,53.0,5501000,47279,0,3000,5
1,1,4586,815000,4610,2,2,1,93.0,5501000,47279,0,3000,5
2,2,4857,3350000,26527,3,2,2,175.0,10721000000,47279,0,3000,5
3,3,4857,3350000,26527,3,2,2,175.0,10721000000,47279,0,3000,5
4,4,771,590000,32788,2,1,1,80.0,5501000,47279,0,3000,5


In [17]:
from sklearn.model_selection import train_test_split

x = df[['bedrooms', 'bathrooms', 'parking', 'area', 'state']]
y = df['price']

x_train, x_test, y_train, y_test = train_test_split(x, y, random_state=2345)

In [18]:
from catboost import CatBoostRegressor

model = CatBoostRegressor()
model.fit(x_train, y_train)

Learning rate set to 0.083319
0:	learn: 593386.8996400	total: 55.9ms	remaining: 55.9s
1:	learn: 581145.7741891	total: 61.1ms	remaining: 30.5s
2:	learn: 570531.3567251	total: 67.8ms	remaining: 22.5s
3:	learn: 560303.0885977	total: 73.1ms	remaining: 18.2s
4:	learn: 551431.4506774	total: 78.8ms	remaining: 15.7s
5:	learn: 544098.0424354	total: 84.4ms	remaining: 14s
6:	learn: 537949.6301658	total: 90ms	remaining: 12.8s
7:	learn: 532073.7261762	total: 94.8ms	remaining: 11.8s
8:	learn: 527548.2386269	total: 100ms	remaining: 11.1s
9:	learn: 522869.5001375	total: 105ms	remaining: 10.4s
10:	learn: 518529.8084282	total: 110ms	remaining: 9.93s
11:	learn: 515098.5150966	total: 115ms	remaining: 9.49s
12:	learn: 511384.0074068	total: 120ms	remaining: 9.11s
13:	learn: 508352.9999221	total: 124ms	remaining: 8.77s
14:	learn: 505654.4718978	total: 129ms	remaining: 8.45s
15:	learn: 503239.9199923	total: 133ms	remaining: 8.15s
16:	learn: 500612.3446529	total: 136ms	remaining: 7.88s
17:	learn: 498200.969617

In [19]:
from sklearn.metrics import r2_score

r2_score(y_test, model.predict(x_test))

0.4914885068193938

In [20]:
from xgboost import XGBRegressor

model2 = XGBRegressor()
model2.fit(x_train, y_train)

XGBRegressor(base_score=None, booster=None, callbacks=None,
             colsample_bylevel=None, colsample_bynode=None,
             colsample_bytree=None, device=None, early_stopping_rounds=None,
             enable_categorical=False, eval_metric=None, feature_types=None,
             gamma=None, grow_policy=None, importance_type=None,
             interaction_constraints=None, learning_rate=None, max_bin=None,
             max_cat_threshold=None, max_cat_to_onehot=None,
             max_delta_step=None, max_depth=None, max_leaves=None,
             min_child_weight=None, missing=nan, monotone_constraints=None,
             multi_strategy=None, n_estimators=None, n_jobs=None,
             num_parallel_tree=None, random_state=None, ...)

In [21]:
r2_score(y_test, model2.predict(x_test))

0.48766753946111996

In [22]:
# import pickle

# with open('./model.pkl', 'wb') as file:
#     pickle.dump(model, file)

In [23]:
prediction = model.predict([[1, 2, 1,2700, 2]])
result = prediction[0] 

In [24]:
result

614382.7981597027

In [25]:
le_state.classes_

array(['New South Wales', 'Northern Territory', 'Queensland',
       'Southern Australia', 'Tasmania', 'Victoria', 'Western Australia'],
      dtype=object)

In [26]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 119644 entries, 0 to 119643
Data columns (total 13 columns):
 #   Column        Non-Null Count   Dtype  
---  ------        --------------   -----  
 0   Unnamed: 0    119644 non-null  int64  
 1   date          119644 non-null  int64  
 2   price         119644 non-null  int64  
 3   address       119644 non-null  int64  
 4   bedrooms      119644 non-null  int64  
 5   bathrooms     119644 non-null  int64  
 6   parking       119644 non-null  int64  
 7   area          119644 non-null  float64
 8   median_price  119644 non-null  int64  
 9   population    119644 non-null  int64  
 10  average_age   119644 non-null  int64  
 11  zipcode       119644 non-null  int64  
 12  state         119644 non-null  int64  
dtypes: float64(1), int64(12)
memory usage: 11.9 MB


In [27]:
from sklearn.ensemble import RandomForestRegressor
model_rf=RandomForestRegressor()
model_rf.fit(x_train,y_train)

RandomForestRegressor()

In [28]:
from sklearn.metrics import r2_score
r2=r2_score(y_test,model_rf.predict(x_test))
r2

0.4155260702815047

In [ ]:
from sklearn.model_selection import GridSearchCV
from catboost import CatBoostRegressor

# Define the parameter grid
param_grid = {
    'iterations': [100, 200, 300],
    'learning_rate': [0.01, 0.03, 0.1],
    'depth': [4, 6, 8, 10],
    'l2_leaf_reg': [1, 3, 5, 7],
    'bagging_temperature': [0.5, 1, 2],
    'random_strength': [0.5, 1, 2]
}

# Initialize the CatBoostRegressor
model = CatBoostRegressor(verbose=0)

# Initialize GridSearchCV
grid_search = GridSearchCV(estimator=model, param_grid=param_grid, scoring='r2', cv=5, n_jobs=-1, verbose=1)

# Fit GridSearchCV
grid_search.fit(x_train, y_train)

# Get the best parameters
best_params = grid_search.best_params_

# Print the best parameters
print("Best parameters found: ", best_params)

# Train the model with the best parameters
best_model = CatBoostRegressor(**best_params, verbose=0)
best_model.fit(x_train, y_train)

# Evaluate the model
r2 = r2_score(y_test, best_model.predict(x_test))
print("Improved R² score: ", r2)


Fitting 5 folds for each of 1296 candidates, totalling 6480 fits
